## LLM Emotion + Likeness Rating & Explanation for Paintings

### Model information: Gemini 2.5 flash
### Two prompts are used, a simple prompt and an enhanced prompt with rating score anchoring + explicitly asking the model to pay attention to low level features
### output "emotion_output.csv" from simple prompt and "emotion_output_enhanced.csv" for enhanced prompt

In [2]:
#import libraries

from google import genai
from google.genai import types
from google.api_core import exceptions as google_exceptions


import os
import re
import time
import random
import logging, sys
from pathlib import Path
import mimetypes

import pandas as pd
from tqdm import tqdm
from PIL import Image
from functools import reduce


%run "api_key.ipynb" #import API key as needed


logging.basicConfig(
    stream=sys.stdout,             
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S")

#### For each emotion, write one model to produce output. Plus one aesthetic rating model.

In [13]:
# ——— CONFIGURATION ———————————————————————————————————————————————————————————————————
IMAGE_FOLDER      = Path("/Users/Stella/Desktop/EmotionArt/emotion-art/data/40_test")
OUTPUT_CSV        = Path("emotion_model_output_vertex.csv")
MODEL_NAME        = "gemini-2.5-flash"
MAX_RETRIES       = 6
BACKOFF_BASE      = 5.0
BATCH_SIZE        = 10
MAX_OUTPUT_TOKENS = 2049
IMAGE_EXTS        = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff"}

In [14]:
# ——— LOGGING ———————————————————————————————————————————————————————————————————————
logging.basicConfig(
    stream=sys.stdout,             
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S"
)

In [15]:
# ——— HELPER FUNCTIONS —————————————————————————————————————————————————————————————————
def get_mime_type(path: str) -> str:
    mime, _ = mimetypes.guess_type(path)
    return mime or "application/octet-stream"

def is_image_file(path: Path) -> bool:
    return path.is_file() and path.suffix.lower() in IMAGE_EXTS and not path.name.startswith(".")

_SCORE_RE = re.compile(r"^[A-Za-z]+:\s*([\d.]+)", re.MULTILINE)
_EXPL_RE  = re.compile(r"Explanation:\s*(.*)", re.DOTALL)

def parse_response(raw: str) -> tuple[float|None, str]:
    text = raw.decode("utf-8", errors="ignore") if isinstance(raw, (bytes, bytearray)) else str(raw or "")
    m_score = _SCORE_RE.search(text)
    m_expl  = _EXPL_RE.search(text)
    if not (m_score and m_expl):
        return None, text.strip()
    return float(m_score.group(1)), m_expl.group(1).strip()

#for batch processing
def chunker(seq, size):
    for i in range(0, len(seq), size):
        yield seq[i : i + size]


In [16]:
#prompts
def build_emotion_prompt(emotion: str) -> str:
    return (
        f"You are an art expert describing your emotional response to a painting.\n"
        f"Evaluate **{emotion.lower()}** independently, without reference to any other feeling. "
        "Do not assume anything about other possible emotional reactions — focus only on this one emotion.\n\n"
        "Provide your response using the following structure:\n"
        "1. A **numeric score** between 0 and 100 (on a continuous scale — do not round to nearest 5 or 10 unless warranted)\n"
        "2. A **detailed explanation** supporting the reason behind the rating you provided. Please try to be as detailed as possible.\n\n"
        f"Use the format exactly:\n"
        f"{emotion}: [score]\n"
        "Explanation: ..."
    )

# Prompt dictionary for loop call models later on with all emotions and liking rating:
prompt_dict = {emotion: build_emotion_prompt(emotion) for emotion in ["Joy", "Sadness", "Fear", "Anger", "Disgust", "Surprise"]}
prompt_dict["Liking"] = (
    "You are an art expert evaluating how much you like a painting.\n"
    "Rate your **personal aesthetic preference** for the painting, based only on what is visually presented.\n"
    "Provide your response using the following structure:\n"
    "1. A **numeric score** between 0 and 100 (on a continuous scale — do not round unless appropriate)\n"
    "2. A **detailed explanation** supporting the reason behind the rating you provided. Please try to be as detailed as possible.\n\n"
    "Use the format exactly:\n"
    "Liking: [score]\n"
    "Explanation: ..."
)


In [17]:
# Enhanced prompt that defines a clear scale anchors (0, 50, 100)
def build_emotion_prompt_enhanced(emotion: str) -> str:
    return (
        "You are an expert art critic and psychologist, trained to assess a viewer's emotional response"
        f"to a painting. Focus **only** on **{emotion.lower()}**—do not blend in any other feeling.\n\n"
        "**Scale definition (0-100):**  \n"
        f"- **0** means “no sense of {emotion.lower()} at all.”  \n"
        "- **50** means “a moderate, everyday level—what most people might feel in a typical scene.”  \n"
        f"- **100** means “an overwhelming, emotionally extreme sense of {emotion.lower()}.”\n\n"
        "**Instructions:**  \n"
        "1. Look closely at composition, color palette, lighting, brushwork, subject matter, and style.  \n"
        "2. Compare what you see to the anchors above—if it's slightly more than “everyday,” pick something like 60-70; if it barely registers, choose 5-10.  \n"
        f"3. Avoid clustering at 50: if the painting truly feels neutral for “{emotion.lower()},” explain why and use exactly 50; otherwise pick a number that reflects the visual evidence.  \n"
        "4. If you choose above 85 or below 15, you must justify why it crosses into “extreme” territory.  \n"
        "5. **Write exactly five complete sentences** in your explanation—no more, no fewer.  \n\n"
        "Provide your response using the following structure:\n"
        "1. A **numeric score** between 0 and 100 (on a continuous scale — do not round unless appropriate)\n"
        "2. A **detailed explanation** A detdescription of the visual elements (e.g., “the high-contrast reds and jagged lines give a surge of …”) that led you to that score. \n\n"
        "**Output format (exactly):**  \n"
        f"{emotion}: [score]\n"
        "Explanation: ..."
    )

prompt_dict_enhanced = {
    emotion: build_emotion_prompt_enhanced(emotion)
    for emotion in ["Joy", "Sadness", "Fear", "Anger", "Disgust", "Surprise"]
}

prompt_dict_enhanced["Liking"] = (
    "You are an expert art critic rating your own **aesthetic preference** for a painting on a 0-100 scale.\n\n"
    "**Scale definition (0-100):**  \n"
    "- **0** means “I wouldn't want this in my home or collection.”  \n"
    "- **50** means “it's average—interesting but not memorable.”  \n"
    "- **100** means “I find it utterly compelling and would absolutely display it.”\n\n"
    "**Instructions:**  \n"
    "1. Consider composition, color harmony, technique, originality, and emotional impact on *you*.  \n"
    "2. Anchor your number to the scale above—if your preference is tepid, choose 30-40; if you love it, choose 80-95.  \n"
    "3. Avoid mid-range clustering—only use 50 if it truly feels neutral.  \n"
    "4. If you go above 90 or below 10, explain why it's so extremely likable or unlikable.  \n"
    "5. **Write exactly five complete sentences** in your explanation—no more, no fewer.  \n\n"
    "Provide your response using the following structure:\n"
    "1. A **numeric score** between 0 and 100 (on a continuous scale — do not round unless appropriate)\n"
    "2. A **detailed explanation** supporting the reason behind the rating you provided. Please try to be as detailed as possible.\n\n"
    f"**Output format (exactly):**  \n"
    "Liking: [score]\n"
    "Explanation: ..."
)


In [18]:
# ——— PRELOAD IMAGES AND PROMPTS ——————————————————————————————————————————————————————————
all_images  = sorted(p for p in IMAGE_FOLDER.iterdir() if is_image_file(p))

# ——— VERTEX AI CLIENT FACTORY —————————————————————————————————————————————————————————
def make_vertex_client():
    return genai.Client(
        vertexai=True,
        project="emotion-art-analysis",
        location="global",
    )


FileNotFoundError: [Errno 2] No such file or directory: '/Users/Stella/Desktop/EmotionArt/emotion-art/data/40_test'

In [19]:
# ——— CORE EMOTION CALL —————————————————————————————————————————————————————————————————
def emotion_model_vertex(image_path: str, prompt_text: str, emotion_label: str, client) -> dict:
    with open(image_path,"rb") as img_file:
        image_bytes = img_file.read()
        
    mime_type = get_mime_type(image_path)

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = client.models.generate_content(
                model   = MODEL_NAME,
                contents = [
                    types.Part.from_bytes(data=image_bytes, mime_type=mime_type),
                    types.Part.from_text(text=prompt_text),
                ],
                config = types.GenerateContentConfig(
                    max_output_tokens=MAX_OUTPUT_TOKENS
                )
            )

            raw = resp.candidates[0].content.parts[0].text if resp.candidates else ""
            score, explanation = parse_response(raw)
            return {
                "image": Path(image_path).name,
                f"{emotion_label.lower()}_rating":      score,
                f"{emotion_label.lower()}_explanation": explanation
            }

        except google_exceptions.ServiceUnavailable:
            backoff = min(BACKOFF_BASE * 2 ** (attempt - 1), 30) + random.random()
            logging.warning(f"503 overload (try {attempt}), sleeping {backoff:.1f}s")
            time.sleep(backoff)

        except Exception as e:
            backoff = min(BACKOFF_BASE * 2 ** (attempt - 1), 30) + random.random()
            logging.warning(f"Error on try {attempt}: {e}")
            time.sleep(backoff)

    return {
        "image": image_path.name,
        f"{emotion_label.lower()}_rating":      None,
        f"{emotion_label.lower()}_explanation": f"ERROR: model overloaded after {MAX_RETRIES} tries"
    }


In [20]:
# ——— BATCHED WRAPPER —————————————————————————————————————————————————————————————————
def get_response_by_emotion_vertex(emotion_label: str, prompt_dict, batch_size: int = BATCH_SIZE):
    client = make_vertex_client()
    records = []
    total   = len(all_images)
    total_batches = (total+batch_size-1) // batch_size

    for b_idx, batch in enumerate(chunker(all_images, batch_size), start=1):
        logging.info(f"Starting {emotion_label} batch {b_idx}/{total_batches} (size={len(batch)})")
        for i, img_path in enumerate(batch, start=1):
            idx = (b_idx - 1) * batch_size + i
            name = img_path.name
            logging.info(f"[{idx}/{total}] {emotion_label}: Processing {name}")

            try:
                rec = emotion_model_vertex(str(img_path), prompt_dict[emotion_label], emotion_label,client)
                logging.info(f"→ {emotion_label} Success: {name} → rating={rec[f'{emotion_label.lower()}_rating']}")
            except Exception as e:
                logging.error(f"❌ {emotion_label} error on {name}: {e}", exc_info=True)
                rec = {
                    "image":               name,
                    f"{emotion_label.lower()}_rating":      "",
                    f"{emotion_label.lower()}_explanation": f"ERROR: {e}"
                }

            records.append(rec)
    df = pd.DataFrame(records)
    #df.to_csv(OUTPUT_CSV.with_stem(f"{emotion_label.lower()}_vertex"), index=False)
    print(f"✅ {emotion_label} (Vertex) results saved to {df.shape[0]} rows")
    return df


### Process 50 images test

In [23]:
joy_50 = get_response_by_emotion_vertex("Joy", prompt_dict)
joy_50.to_csv("../output/joy_50.csv", index=False)

✅ Joy (Vertex) results saved to 50 rows


In [24]:
sadness_50 = get_response_by_emotion_vertex("Sadness", prompt_dict)
sadness_50.to_csv("../output/sadness_50.csv", index=False)

✅ Sadness (Vertex) results saved to 50 rows


In [25]:
fear_50 = get_response_by_emotion_vertex("Fear", prompt_dict)
fear_50.to_csv("../output/fear_50.csv", index=False)


23:16:40 [WARNING] Error on try 1: 'NoneType' object is not subscriptable
✅ Fear (Vertex) results saved to 50 rows


In [26]:
anger_50 = get_response_by_emotion_vertex("Anger", prompt_dict)
anger_50.to_csv("../output/anger_50.csv", index=False)


23:30:16 [WARNING] Error on try 1: 'NoneType' object is not subscriptable
23:30:42 [WARNING] Error on try 2: 'NoneType' object is not subscriptable
✅ Anger (Vertex) results saved to 50 rows


In [27]:
disgust_50 = get_response_by_emotion_vertex("Disgust", prompt_dict)
disgust_50.to_csv("../output/disgust_50.csv", index=False)


✅ Disgust (Vertex) results saved to 50 rows


In [28]:
surprise_50 = get_response_by_emotion_vertex("Surprise", prompt_dict)
surprise_50.to_csv("../output/surprise_50.csv", index=False)

23:45:17 [WARNING] Error on try 1: 'NoneType' object is not subscriptable
✅ Surprise (Vertex) results saved to 50 rows


In [29]:
liking_50 = get_response_by_emotion_vertex("Liking", prompt_dict)
liking_50.to_csv("../output/liking_50.csv", index=False)

✅ Liking (Vertex) results saved to 50 rows


------END-------

### 882 images in total